In [1]:
pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.


In [1]:
api_key="ce745ac8-9f0e-4f4b-9e9c-0b75d66fa28e"

In [2]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [4]:
import os
from pinecone import Pinecone, ServerlessSpec
index_name="hybrid-search-langchain-pinecone"
# initialize the Pinecone client
pc=Pinecone(api_key=api_key)

#create the index

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, # dimension of dense vector
        metric="dotproduct", # Sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )



In [5]:
index=pc.Index(index_name)
index

In [7]:
## Vector embedding and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

c:\Users\Uyama\Downloads\LANGCHAIN-PROJECTS\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [21]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()
bm25_encoder

In [22]:
sentences=[
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2011, I visited New Orleans",
]
#applying tfidf values on these sentences
bm25_encoder.fit(sentences)

#store the values to a json file
bm25_encoder.dump("bm25_values.json")

#load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 2138.86it/s]


In [18]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index=index)

In [23]:
retriever.add_texts([
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2011, I visited New Orleans",
])

100%|██████████| 1/1 [00:01<00:00,  1.71s/it]


In [24]:
retriever.invoke("What city did I last visit")

[Document(metadata={'score': 0.249781087}, page_content='In 2011, I visited New Orleans'),
 Document(metadata={'score': 0.235075161}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.217154011}, page_content='In 2023, I visited Paris')]